# Lista: Interpolação e Integração Numérica

### 1. Interpolação com funções de uma variável

####   (a) Implemente um função em Julia tal que
      i. Entrada: os pontos (x0, y0),(x1, y1), (x2, y2) e (x3, y3).
      ii. Saída: os 4 coeficientes do polinômio P(x) de grau 3 tal que P(xi) = yi

In [118]:
using LinearAlgebra

function interpolação_grau3(pontos)
    # cria a matriz de vandermonde, que contem as constantes que multiplicam as variáveis que queremos achar
    vandermonde =   [1 pontos[1,1] pontos[1,1]^2 pontos[1,1]^3;
                     1 pontos[2,1] pontos[2,1]^2 pontos[2,1]^3;
                     1 pontos[3,1] pontos[3,1]^2 pontos[3,1]^3;
                     1 pontos[4,1] pontos[4,1]^2 pontos[4,1]^3;]
    
    # cria um vetor com os valores de y em cada x
    y = [pontos[1,2]; pontos[2,2]; pontos[3,2]; pontos[4,2]]
    
    # retorna a solução do sistema Vc = y
    # que é um vetor em que cada posição corresponde a um coeficiente do polinômio
    return vandermonde \ y
end

interpolação_grau3 (generic function with 1 method)

Testando a solução

In [119]:
function checka_interpolação(coeficientes, pontos)
    resultado = true
    m, n = size(pontos)
    
    # percorre todas as equações
    for j=1:m
        yi = 0
        xi = pontos[j,1]
        
        # calcula o valor do polinômio em xi
        for i=1:m
            yi += coeficientes[i] * xi^(i-1)
        end
        
        # se algum yi diferir do valor, resultado vai pra false
        resultado = resultado && (yi - pontos[j,2] < 1e-8)
    end
    
    return resultado ? :correto : :errado
end

checka_interpolação (generic function with 1 method)

In [120]:
pontos = [1.0 0; 5 0; 3 0; 0 -15]

coeficientes = interpolação_grau3(pontos)
println(coeficientes)
checka_interpolação(coeficientes, pontos)

[-15.0, 23.0, -9.0, 1.0]


:correto

In [121]:
pontos = [-1.0 0; 0 0; 1 0; 2 6]

interpolação_grau3(pontos)
println(coeficientes)
checka_interpolação(coeficientes, pontos)

[-15.0, 23.0, -9.0, 1.0]


:correto

#### (b) (Bônus) Generalize o item anterior para a função poder receber n pontos.

In [122]:
function interpolação(pontos)
    m, n = size(pontos)
    vandermonde = zeros(m, m)
    y = zeros(m)
    
    # cria a matriz de vandermonde para um polinômio de grau m-1
    for i = 1:m
        y[i] = pontos[i,2]

        for j = 1:m
            vandermonde[i, j] = pontos[i,1]^(j-1)
        end
    end
    
    return vandermonde \ y
end

interpolação (generic function with 1 method)

Testando a solução

In [123]:
pontos = [1.0 -2; 2 2; 3 -1; 4 7;]

coeficientes = interpolação_grau3(pontos)
println(coeficientes)
checka_interpolação(coeficientes, pontos)

[-31.0, 47.5, -21.5, 3.0]


:correto

In [124]:
pontos = [1.0 -6; 2 17; 3 88; 4 105; 5 -274; 6 2000; 5.5 3000]

coeficientes = interpolação(pontos)
println(coeficientes)
checka_interpolação(coeficientes, pontos)

[-144717.42857145894, 353924.98333338427, -324991.99166668655, 146559.20833332953, -34726.79166666282, 4141.808333332555, -195.78809523804517]


:correto

#### (c) Implemente um função em Julia tal que
i. Entrada: os pontos (x0, y0),(x1, y1), (x2, y2) e (x3, y3).

ii. Saída: os 3 coeficientes c0, c1, c2 de $G(x) = c_0sen(x) + c_1x^2 + c_2x^{10}$

tal que $G(x_i) = y_i$

In [125]:
function interpolação_c(pontos)
    # cria a matriz de vandermonde, que contem as constantes que multiplicam as variáveis que queremos achar
    vandermonde = [sin(pontos[1,1]) pontos[1,1]^2 pontos[1,1]^10;
                   sin(pontos[2,1]) pontos[2,1]^2 pontos[2,1]^10;
                   sin(pontos[3,1]) pontos[3,1]^2 pontos[3,1]^10;
                   sin(pontos[4,1]) pontos[4,1]^2 pontos[4,1]^10;]

    # cria um vetor com os valores de y em cada x
    y = [pontos[1,2]; pontos[2,2]; pontos[3,2]; pontos[4,2]]
    
    # retorna a solução do sistema Vc = y
    # que é um vetor em que cada posição corresponde a um coeficiente do polinômio
    return vandermonde \ y
end

interpolação_c (generic function with 1 method)

#### iii. Dá para verificar se deu tudo certo?
A função abaixo checa se todos os valores fornecidos pela função $G(x)$ estão de acordo com os valores passados inicialmente

Como a entrada é com quatro pontos, é possível que alguma das equações torne o sistema sem solução. Entretanto, esse resultado não aparece aqui no notebook, em alguns casos o operador "\" retorna uma solução mesmo que o sistema não tenha nenhuma. Em outros casos, o operador retorna uma "Singular Exception". Dessa forma, o único teste possível seria verificar se a função final interpola de fato todos os pontos passados. Não entendi porque isso acontece em Julia, infelizmente não consegui achar com detalhes o funcionamento do operador. Apesar disso, teoricamente ficaria bem claro quando o conjunto de pontos passados na entrada não pode ser representado como combinação linear das funções, pois o sistema não terá solução.

In [126]:
function checka_interpolação_c(coeficientes, pontos)
    resultado = true
    m, n = size(pontos)
    
    # percorre todas as equações
    for j=1:m
        xi = pontos[j,1]
        
        # calcula o valor do polinômio em xi
        yi = coeficientes[1] * sin(xi) + coeficientes[2] * xi^2 + coeficientes[3] * xi^10
        
        # se algum yi diferir do valor, resultado vai pra false
        resultado = resultado && (yi - pontos[j,2] < 1e-10)
    end

    return resultado ? :correto : :errado
end

checka_interpolação_c (generic function with 1 method)

Testando a solução

In [130]:
f(x) = -3*sin(x) + 2*x^2 - 0.05*x^10

f(0.1), f(0.5), f(0.9), f(1.3)

(-0.27950024994548445, -0.938325443937609, -0.7474146508874503, -0.1999670154965787)

In [131]:
pontos = [0.1 -0.27950024994548445; 0.5 -0.938325443937609; 0.9 -0.7474146508874503; 1.3 -0.1999670154965787]

coeficientes = interpolação_c(pontos)
println(coeficientes)
checka_interpolação_c(coeficientes, pontos)

[-3.0000000000000013, 2.0000000000000018, -0.0500000000000002]


:correto

In [127]:
pontos = [1.0 1; 2 1; 3 1; 4 1]

coeficientes = interpolação_c(pontos)
println(coeficientes)
checka_interpolação_c(coeficientes, pontos)

[0.8882021306834879, 0.08890899570066767, 2.411672428014811e-7]


:errado

### 2. Integração numérica - Regra do Trapézio

(a) Implemente uma função em Julia tal que

• Entrada: uma função f, um número positivo n, um número real a, e um número real b.

• Saída: uma aproximação de $\int_{a}^{b} f(x) \,dx$ com n trapézios.

In [2]:
function integração_trapézios(f, a, b, n)
    h = abs(b - a) / n # calcula o tamanho da partição
    
    # usando a regra composta, a linha abaixo calcula a parte do somatório que não tem coeficiente 2
    # primeiro faço a soma das bases de cada trapézio para no final multiplicar tudo pela altura
    soma_bases = f(a) + f(b) 
    
    # calcula as parcelas restantes do método
    for i = 1:n-1
        soma_bases += 2*f(a + i*h)
    end
    
    # a área final sera multiplicada pelo valor da base do trapézio
    area = soma_bases * h / 2
    
    return area
end

integração_trapézios (generic function with 1 method)

Testando a solução. O resultado deve ser próximo de 0.333...

In [3]:
f(x) = x^2

integração_trapézios(f, 0, 1, 100)

0.33335000000000004

#### (b) Determine uma função f não-linear, a, b, e n > 4 tal que o erro da aproximação é 0 na questão anterior.

Um exemplo é a função f(x)=x^3, para a = -1 e b = 1, a integral dará 0 e pela regra do trapézio, para qualquer n par dará 0 também

Isso acontece porque x^3 é uma função ímpar, então os valores da função num intervalo simétrico irão se cancelar.


#### (c) Use a sua criatividade para criar uma função em Julia para aproximar uma integral dupla $\int_{a}^{b}\int_{c}^{d} f(x) \,dx$ numéricamente (não tem resposta certa, mas você precisa justificar o que você fez).

A solução abaixo generaliza a regra para qualquer função contínua R^2 -> R

Ele utiliza o mesmo princípio da regra dos trapézios para uma variável.

Primeiro ele particiona o intervalo [c,d] e, em cada partição, calcula a área em baixo da função, considerando y constante.

Depois disso, multiplica a área pelo tamanho da partição em y.

A geometricamente, o método funciona, não particionando o sólido em em paralelepípedos, mas sim em primas trapezoidais.

In [77]:
function integração_trapézios_integral_dupla(f, a, b, c, d, n, m)
    # calcula o tamanho das partições
    hx = abs(b - a) / n
    hy = abs(d - c) / m
    
    volume = 0
    area_parcial = 0
    
    for i = 0:m
        # torna y contante e calcula a área embaixo da função
        yi = c + i*hy
        
        # usando a regra composta, a linha abaixo calcula a parte do somatório que não tem coeficiente 2
        # primeiro faço a soma das bases de cada trapézio para no final multiplicar tudo pela altura
        soma_bases = f(a, yi) + f(b, yi) # calcula a
        
        # calcula as parcelas restantes do método
        for j = 1:n-1
            xi = a + j*hx
            
            soma_bases += 2*f(xi, yi)
        end
        
        # a área embaixo da função será a soma das bases vezes a altura do trapézio sobre 2
        area_parcial = soma_bases * hx / 2
        
        # o volume será a área parcial de cada partição de y vezes o tamanho da partição

        volume += area_parcial * hy
    end
    
    return volume
end

integração_trapézios_integral_dupla (generic function with 1 method)

Testando a solução, o resultado deve ser próximo de 1.16

In [78]:
f(x,y) = x*y^2

integração_trapézios_integral_dupla(f, 0, 1, 1, 2, 2000, 2000)

1.167291687500001

### 3. Interpolação para funções de duas variáveis.

In [79]:
function checka_interpolação(coeficientes, pontos)
    resultado = true
    m, n = size(pontos)
    
    # percorre todas as equações
    for j=1:m
        xi = pontos[j,1]
        yi = pontos[j,2]
        
        # calcula o valor do polinômio em xi
        zi = coeficientes[1] * xi*yi + coeficientes[2] * xi + coeficientes[3] * yi + coeficientes[4]

        # se algum yi diferir do valor, resultado vai pra false
        resultado = resultado && (zi - pontos[j,3] < 1e-10)
    end

    return resultado ? :correto : :errado
end

checka_interpolação (generic function with 1 method)

In [80]:
pontos = [1.0 2 700; 1 4 500; 3 2 500; 3 4 600]

coeficientes = interpolação_montanha(pontos)
println(coeficientes)
checka_interpolação(coeficientes, pontos)

UndefVarError: UndefVarError: interpolação_montanha not defined

#### (a) Dada algumas medições da tabela da função altura A(x, y) de uma montanha em posições diferentes (x0, y0),(x0, y1),(x1, y0),e (x1, y1) determine a melhor aproximação possível para a posição e altura do pico ou vale (máximo e mínimos locais) se existir da montanha usando a interpolação quadrática para funções de duas variáveis que vimos na aula.

Podemos fazer essa interpolação usando o polinômio de Lagrange

$$P(x,y) = A(x_0,y_0) \cdot \frac{(x - x_1)(y - y_1)}{(x_0 - x_1)(y_0 - y_1)} + A(x_0,y_1) \cdot \frac{(x - x_1)(y - y_0)}{(x_0 - x_1)(y_1 - y_0)} + A(x_1,y_0) \cdot \frac{(x - x_0)(y - y_1)}{(x_1 - x_0)(y_0 - y_1)} + A(x_1,y_1) \cdot \frac{(x - x_0)(y - y_0)}{(x_1 - x_0)(y_1 - y_0)}$$

Substituindo os pontos fornecidos na tabela temos



$$P(x,y) = A(1,2) \cdot \frac{(x - 3)(y - 4)}{(1 - 3)(2 - 4)} + A(1,4) \cdot \frac{(x - 3)(y - 2)}{(1 - 3)(4 - 2)} + A(3,2) \cdot \frac{(x - 1)(y - 4)}{(3 - 1)(2 - 4)} + A(3,4) \cdot \frac{(x - 1)(y - 2)}{(3 - 1)(4 - 2)}$$



$$P(x,y) = 700 \cdot \frac{(x - 3)(y - 4)}{(1 - 3)(2 - 4)} + 500 \cdot \frac{(x - 3)(y - 2)}{(1 - 3)(4 - 2)} + 500 \cdot \frac{(x - 1)(y - 4)}{(3 - 1)(2 - 4)} + 600 \cdot \frac{(x - 1)(y - 2)}{(3 - 1)(4 - 2)}$$



$$P(x, y) = 75 \cdot xy - 250 \cdot x - 175 \cdot y + 1150$$



Os máximos e mínimos da superfície estarão nos pontos críticos ou na borda do retangulo formado pelos intervalos $[1,3] \times [2,4]$


Para achar os pontos críticos, precisamos das derivadas parciais:

$$\frac{\partial}{\partial x}P(x,y) = 75 \cdot y - 250$$

$$\frac{\partial}{\partial y}P(x,y) = 75 \cdot x - 175$$

O único ponto crítico, quando a derivada é zero, é em $(\frac{7}{3}, \frac{10}{3})$. Nesse ponto $P(\frac{7}{3}, \frac{10}{3}) = 5100/9 = 566,6...$

Os valores nas bordas do intervalo são

Fixando y = 2
$$f_1(x) = P(x, 2) = -100 \cdot x + 800$$

O máximo da função, para $1 \leq x \leq 3$, é $f_1(1) = P(1,2) = 700$, quando $x = 1$. O mínimo é $f_1(3) = P(3,2) = 500$

Fixando y = 4
$$f_2(x) = P(x, 4) = 50 \cdot x + 450$$

O máximo da função, para $1 \leq x \leq 3$, é $f_2(3) = P(3,4) = 600$, quando $x = 1$. O mínimo é $f_2(1) = P(1,4) = 500$

Fixando x = 1
$$f_3(y) = P(1, y) = -100 \cdot y + 900$$

O máximo da função, para $2 \leq y \leq 4$, é $f_3(2) = P(1,2) = 700$, quando $y = 2$. O mínimo é $f_3(4) = P(1,4) = 500$

Fixando x = 3
$$f_4(y) = P(3, y) = 50 \cdot y + 400$$

O máximo da função, para $2 \leq y \leq 4$, é $f_4(4) = P(3,4) = 600$, quando $y = 2$. O mínimo é $f_4(2) = P(3,2) = 500$


Dessa forma podemos afirmar que os vales da montanha são em $(3,2)$ e $(1,4)$
E o pico é em $(1,2)$